In [1]:
import os

import numpy as np
import pandas as pd
import numpy.testing as npt
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

import sys
sys.path.append('..')

import modules.stats as st
import modules.plotting as pl
import modules.pandas_funcs as pf

In [2]:
% matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
results_dir = os.path.join('..', 'data', 'results')
match_dir = os.path.join('..', 'data', 'matching')


df_k_raw = pd.read_csv(os.path.join(results_dir, 'kinect_gait_metrics.csv'),
                       index_col=0)

df_z_raw = pd.read_csv(os.path.join(results_dir, 'zeno_gait_metrics.csv'),
                       index_col=0)

df_match = pd.read_csv(os.path.join(match_dir, 'match_kinect_zeno.csv'))

# Drop rows where file has no match
df_match = df_match.dropna(axis=0)


df_match_zeno = pd.merge(df_match, df_z_raw, left_on='Zeno', right_index=True)

df_total = pd.merge(df_match_zeno, df_k_raw, left_on='Kinect',
                    right_index=True, suffixes=('_z', '_k'))


# Take columns from total DataFrame to get Kinect and Zeno data
df_k = df_total.filter(like='_k')
df_z = df_total.filter(like='_z')

# Remove suffixes from column names
df_k = df_k.rename(columns=lambda x: str(x)[:-2])
df_z = df_z.rename(columns=lambda x: str(x)[:-2])


In [4]:
df_error = abs((df_k - df_z) / df_z) * 100

In [5]:
df_error.median()

step_length_L         8.222381
step_length_R        12.797947
stride_length_L      11.052632
stride_length_R       9.934793
stride_time_L        19.966687
stride_time_R        19.219244
stride_velocity_L    10.707527
stride_velocity_R     8.541332
stride_width_L       39.961704
stride_width_R       40.835050
dtype: float64

In [6]:
np.median(df_error.values)

12.802880747697035

In [7]:
# abs_step_length_L = np.sqrt(df_z.step_length_L ** 2 + df_z.stride_width_L ** 2)
# abs(abs_step_length_L - df_z.absolute_step_length_L).max()

In [8]:
# abs_step_length_L = np.sqrt(df_k.step_length_L ** 2 + df_k.stride_width_L ** 2)
# abs(abs_step_length_L - df_k.absolute_step_length_L).idxmax()

In [7]:
max_median_index = df_error.median(axis=1).idxmax()

In [8]:
metric = 'stride_length_R'

In [9]:
plt.figure()
df_error[metric].plot.bar()
plt.show()

In [10]:
max_index = df_error[metric].idxmax()

In [11]:
df_compare = pd.concat([df_k.loc[max_index], df_z.loc[max_index]], axis=1)
df_compare.columns = ['Kinect', 'Zeno']
df_compare

,Kinect,Zeno
step_length_L,38.995410,45.897
step_length_R,42.948636,49.820
stride_length_L,80.575801,97.098
stride_length_R,83.504607,103.696
stride_time_L,1.583333,1.778
stride_time_R,1.566667,1.929
stride_velocity_L,51.239075,54.669
stride_velocity_R,52.320600,53.500
stride_width_L,11.953504,10.281
stride_width_R,12.943304,11.492


In [12]:
df_match.loc[max_index]

Zeno      A_2014-12-08_P007_Post_NC_DT3s
Kinect          2014-12-08_P007_Post_002
Name: 17, dtype: object